### 00 - Importation des librairies

In [2]:
import pandas as pd
import numpy as np
import re

from collections import defaultdict
from datetime import datetime

from utils import racine_projet

In [11]:
racine_projet()

'/Users/kfranceschi/GitHub/projet-pompiers'

### 01 - Importation des tableaux de données Incidents

In [9]:
# Premier Tableau
dtype_dict = {'IncidentNumber': str}
incident_9_17 = pd.read_csv(racine_projet()+'/data/raw/Incident/LFB Incident data from 2009 - 2017.csv', dtype=dtype_dict, low_memory=False)

In [12]:
# Deuxième Tableau
incident_18_24 = pd.read_excel(racine_projet()+'/data/raw/Incident/LFB Incident data from 2018 onwards.csv.xlsx')

In [13]:
# Affichage des différences de type de colonnes
diff_columns = []
for column in incident_9_17.columns:
    if column in incident_18_24.columns:
        if incident_9_17[column].dtype != incident_18_24[column].dtype:
            diff_columns.append(column)
print("Avant traitement les colonnes différentes sont les suivantes:")
print(diff_columns)

# Harmoniser les types de données

# Conservation des types de 9_17
incident_18_24['UPRN']=incident_18_24['UPRN'].astype(incident_9_17['UPRN'].dtype)
incident_18_24['USRN']=incident_18_24['USRN'].astype(incident_9_17['USRN'].dtype)

# Conservation des types de 18_24
for column in incident_18_24.columns:
    if column in incident_9_17.columns:
        # Définir le type de données de incident_9_17 en fonction de incident_18_24
        incident_9_17[column] = incident_9_17[column].astype(incident_18_24[column].dtype)

# Affichage des différences de type de colonnes
diff_columns = []
for column in incident_9_17.columns:
    if column in incident_18_24.columns:
        if incident_9_17[column].dtype != incident_18_24[column].dtype:
            diff_columns.append(column)
print("Après traitement les colonnes différentes sont les suivantes:")
print(diff_columns)

Avant traitement les colonnes différentes sont les suivantes:
['DateOfCall', 'UPRN', 'USRN']


/var/folders/rg/8x31xcvn0hddrq5cpz770vhc0000gn/T/ipykernel_8451/664238661.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  incident_9_17[column] = incident_9_17[column].astype(incident_18_24[column].dtype)


Après traitement les colonnes différentes sont les suivantes:
[]


### 02 - Concatenation incidents

In [14]:
# Concatenation des deux tableaux
incidents = pd.concat([incident_9_17, incident_18_24], ignore_index=True)

### 03 - Importation des tableaux de données mobilisation

In [15]:
mobilisation_9_14 = pd.read_excel(racine_projet()+'/data/raw/Mobilisation/LFB Mobilisation data from January 2009 - 2014.xlsx')

In [16]:
mobilisation_15_20 = pd.read_excel(racine_projet()+'/data/raw/Mobilisation/LFB Mobilisation data from 2015 - 2020.xlsx')

In [17]:
mobilisation_21_24 = pd.read_excel(racine_projet()+'/data/raw/Mobilisation/LFB Mobilisation data 2021 - 2024.xlsx')

In [18]:
# Harmoniser les types de données
for column in mobilisation_15_20.columns:
    if column in mobilisation_9_14.columns:
        mobilisation_9_14[column] = mobilisation_9_14[column].astype(mobilisation_15_20[column].dtype)
    if column in mobilisation_21_24.columns:
        mobilisation_21_24[column] = mobilisation_21_24[column].astype(mobilisation_15_20[column].dtype)
        

### 04 - Concatenation des tableaux de données mobilisation

In [19]:
# Concatenation 
mob = pd.concat([mobilisation_9_14, mobilisation_15_20, mobilisation_21_24 ], ignore_index=True)

### 05 - Merge des tableaux incidents et mobilisation

#### SUPRESSION .00 à la fin des incidents number du tableau INCIDENTS

In [20]:
incidents['IncidentNumber'] = incidents['IncidentNumber'].apply(lambda x: str(x).rstrip('.00') if str(x).endswith('.00') else str(x))

#### TRANSFORMATION du type du numéro en int64

In [21]:
def keep_only_digits(value):
    return re.sub(r'\D', '', str(value))

incidents['IncidentNumber'] = incidents['IncidentNumber'].apply(keep_only_digits)
mob['IncidentNumber'] = mob['IncidentNumber'].apply(keep_only_digits)

incidents['IncidentNumber'] = incidents['IncidentNumber'].astype('int64')
mob['IncidentNumber'] = mob['IncidentNumber'].astype('int64')


#### SUPPRESSION des valeurs non communes aux deux tableaux (sur la base du incident number)

In [22]:
# Affichages des valeurs communes
col1 = incidents['IncidentNumber']
col2 = mob['IncidentNumber']
set1 = set(col1)
set2 = set(col2)

common_values = set1.intersection(set2)
num_common_values = len(common_values)

unique_values = set1.symmetric_difference(set2)

unique_to_df1 = set1 - set2
unique_to_df2 = set2 - set1

print("AVANT TRAITEMENT")
print(f"Nombre de valeurs communes: {num_common_values}")
print(f"Nombre de valeurs non communes: {len(unique_values)}")
print(f"Nombre de valeurs uniques à inter: {len(unique_to_df1)}")
print(f"Nombre de valeurs uniques à mob: {len(unique_to_df2)}")

# Suppression des valeurs communes
incidents = incidents[incidents['IncidentNumber'].isin(common_values)]
mob = mob[mob['IncidentNumber'].isin(common_values)]

col1 = incidents['IncidentNumber']
col2 = mob['IncidentNumber']
set1 = set(col1)
set2 = set(col2)

common_values = set1.intersection(set2)
num_common_values = len(common_values)

unique_values = set1.symmetric_difference(set2)

# Affichages des valeurs communes
unique_to_df1 = set1 - set2
unique_to_df2 = set2 - set1
print("APRES TRAITEMENT")
print(f"Nombre de valeurs communes: {num_common_values}")
print(f"Nombre de valeurs non communes: {len(unique_values)}")
print(f"Nombre de valeurs uniques à inter: {len(unique_to_df1)}")
print(f"Nombre de valeurs uniques à mob: {len(unique_to_df2)}")

AVANT TRAITEMENT
Nombre de valeurs communes: 1569258
Nombre de valeurs non communes: 138822
Nombre de valeurs uniques à inter: 132389
Nombre de valeurs uniques à mob: 6433
APRES TRAITEMENT
Nombre de valeurs communes: 1569258
Nombre de valeurs non communes: 0
Nombre de valeurs uniques à inter: 0
Nombre de valeurs uniques à mob: 0


#### FUSION des 2 TABLEAUX

In [23]:
merged_df = pd.merge(mob, incidents, on='IncidentNumber', how='inner')

#### SUPPRESSION DES DOUBLONS IMMEDIATS

In [24]:
# On vérifie que les colonnes présentes dans les deux tableaux sont bien des doublons
dif1 = merged_df['CalYear_y']-merged_df['CalYear_x']
dif2 = merged_df['HourOfCall_y']-merged_df['HourOfCall_x']
print(dif1.value_counts())
print(dif2.value_counts())

0    2365940
Name: count, dtype: int64
0    2365940
Name: count, dtype: int64


In [25]:
# On peut supprimer les colonnes doublons
merged_df = merged_df.drop(labels = ['CalYear_x','HourOfCall_x'], axis = 1)
merged_df = merged_df.rename(columns = {'CalYear_y':'CalYear','HourOfCall_y':'HourOfCall'})

### 06 - Enregistrement du tableau Master

In [26]:
# Export du nouveau tableau sous format csv
merged_df.to_csv(racine_projet()+'/data/raw/merged_data.csv', index=False)